In [5]:
import numpy as np
import pandas as pd
from mod17 import MOD17
from datetime import datetime
from statsmodels.tsa.api import ExponentialSmoothing

# Calculando GPP de 2002 até 2021 com os parametros otimizados

## Santarem

In [7]:
def constructor_2(path):
    nome = path.split('/')[1].split('_')[0]
    df = pd.read_csv(path, parse_dates=['date'])
    df = df.rename({'albedo':nome},axis=1)
    # Definindo a coluna 'system:time_start' como índice
    hora_inicio = df['date'].min().floor('d')
    df.index = hora_inicio + pd.to_timedelta(df.index, unit='h')
    # Removendo a coluna 'system:time_start'
    df = df.drop(['date','.geo'], axis=1)
    df = df.drop('system:index',axis=1)
    return df

# Leitura e pré-processamento dos dados de temperatura
tday = constructor_2('dados/Merra/santarem/st_t10m.csv') - 273.15 #( K° -> C°)
tday = tday['2002':'2021']
tmin = tday['T10M'].resample('D').min().values  # Reamostrando para Temperatura mínima diária
tmean_santarem = tday['T10M'].resample('D').mean().values  # Temperatura média diária
tmean_vpd = tday['T10M'].between_time('10:00', '22:00').resample('D').mean().values

# Leitura e pré-processamento dos dados de razão de mistura de vapor de água
qv10m = constructor_2('dados/Merra/santarem/st_qv10m.csv') # unidade: "Mass fraction" (kg/kg)
qv10m = qv10m['2002':'2021']
qv10m = pd.to_numeric(qv10m['QV10M']).between_time('10:00', '22:00').resample('D').mean().values

# Leitura e pré-processamento dos dados de pressão atmosférica
ps = constructor_2('dados/Merra/santarem/st_ps.csv') # unidade: Pa  # Removendo vírgulas dos valores
ps = ps['2002':'2021']
ps['PS'] = pd.to_numeric(ps['PS'])
ps = ps.between_time('10:00', '22:00').resample('D').mean()
ps = pd.to_numeric(ps['PS']).values

# Leitura e pré-processamento dos dados de radiação solar incidente
SWGNT = constructor_2('dados/Merra/santarem/st_swgnt.csv')
SWGNT = SWGNT['2002':'2021']
SWGNT = SWGNT['SWGNT'].resample('D').mean().values # unidade: W/m^2

# Função para converter datas no formato MODIS
def modis_date(data_str):
    ano = int(data_str[1:5])
    dia_ano = int(data_str[5:])
    data = datetime.strptime('{}-{}'.format(ano, dia_ano), '%Y-%j')
    return data

# Leitura e pré-processamento dos dados de Fração de Absorção de Luz Fotossinteticamente Ativa (FPAR)
fpar = pd.read_csv('dados/Merra/santarem/st_fpar.csv', header=None, na_values='F') # unidade: (%)
fpar.index = fpar[2].apply(modis_date)  # Convertendo a data no formato MODIS

# Arrumando o dataframe
# ================================
fpar = fpar.drop([0,1,2,3,4], axis=1)
fpar.columns = range(len(fpar.columns))
fpar = fpar[144]

# ================================

# Tratamento de dados faltando usando interpolação linear (8 dias -> diário)
fpar = fpar.resample('D').interpolate('linear')['2002':'2021']

fpar = fpar.values

drivers_santarem = [
    fpar[:][...,None],
    tmin[:][...,None],
    MOD17.vpd(qv10m, ps, tmean_vpd)[...,None],
    MOD17.par(SWGNT)[:][...,None]
]
drivers_santarem[2] = np.where(drivers_santarem[2] < 0, 0, drivers_santarem[2]) # VPD < 0 = 0

dict = {'1': np.squeeze(drivers_santarem[0]), '2': np.squeeze(drivers_santarem[1]), '3': np.squeeze(drivers_santarem[2]), '4': np.squeeze(drivers_santarem[3])} 
index_santarem = pd.date_range(start='2002-01-01', end='2021-12-31', freq='D')

df_drivers_santarem = pd.DataFrame(dict, index=index_santarem)

drivers_santarem_mes = []
for i in range(12):
    mes = df_drivers_santarem[df_drivers_santarem.index.month == i+1].to_numpy().T
    drivers_santarem_mes.append(mes)

bplut_mensal_santarem = pd.read_csv('GA_Santarem.csv')
bplut_mensal_santarem = bplut_mensal_santarem[['LUE_Max', 'Tmin_min', 'Tmin_max', 'VPD_min', 'VPD_max']].to_numpy()


for i in range(12):
    previsoes_santarem = MOD17._gpp(bplut_mensal_santarem[i], drivers_santarem_mes[i][0], drivers_santarem_mes[i][1], drivers_santarem_mes[i][2], drivers_santarem_mes[i][3])
    previsoes_santarem = pd.DataFrame(previsoes_santarem,index=index_santarem[index_santarem.month == i+1])
    
   

    #previsoes_santarem_pso = pd.concat([gpp_kgc_8d_2009_teste[ano2009],gpp_kgc_8d_2010_teste[ano2010],gpp_kgc_8d_2011_teste[ano2011]],axis=0)
    if i > 0:
        previsoes_santarem_2002 = pd.concat([previsoes_santarem_2002, previsoes_santarem['2002':'2002']], axis=0)
        previsoes_santarem_2003 = pd.concat([previsoes_santarem_2003, previsoes_santarem['2003':'2003']], axis=0)
        previsoes_santarem_2004 = pd.concat([previsoes_santarem_2004, previsoes_santarem['2004':'2004']], axis=0)
        previsoes_santarem_2005 = pd.concat([previsoes_santarem_2005, previsoes_santarem['2005':'2005']], axis=0)
        previsoes_santarem_2006 = pd.concat([previsoes_santarem_2006, previsoes_santarem['2006':'2006']], axis=0)
        previsoes_santarem_2007 = pd.concat([previsoes_santarem_2007, previsoes_santarem['2007':'2007']], axis=0)
        previsoes_santarem_2008 = pd.concat([previsoes_santarem_2008, previsoes_santarem['2008':'2008']], axis=0)
        previsoes_santarem_2009 = pd.concat([previsoes_santarem_2009, previsoes_santarem['2009':'2009']], axis=0)
        previsoes_santarem_2010 = pd.concat([previsoes_santarem_2010, previsoes_santarem['2010':'2010']], axis=0)
        previsoes_santarem_2011 = pd.concat([previsoes_santarem_2011, previsoes_santarem['2011':'2011']], axis=0)
        previsoes_santarem_2012 = pd.concat([previsoes_santarem_2012, previsoes_santarem['2012':'2012']], axis=0)
        previsoes_santarem_2013 = pd.concat([previsoes_santarem_2013, previsoes_santarem['2013':'2013']], axis=0)
        previsoes_santarem_2014 = pd.concat([previsoes_santarem_2014, previsoes_santarem['2014':'2014']], axis=0)
        previsoes_santarem_2015 = pd.concat([previsoes_santarem_2015, previsoes_santarem['2015':'2015']], axis=0)
        previsoes_santarem_2016 = pd.concat([previsoes_santarem_2016, previsoes_santarem['2016':'2016']], axis=0)
        previsoes_santarem_2017 = pd.concat([previsoes_santarem_2017, previsoes_santarem['2017':'2017']], axis=0)
        previsoes_santarem_2018 = pd.concat([previsoes_santarem_2018, previsoes_santarem['2018':'2018']], axis=0)
        previsoes_santarem_2019 = pd.concat([previsoes_santarem_2019, previsoes_santarem['2019':'2019']], axis=0)
        previsoes_santarem_2020 = pd.concat([previsoes_santarem_2020, previsoes_santarem['2020':'2020']], axis=0)
        previsoes_santarem_2021 = pd.concat([previsoes_santarem_2021, previsoes_santarem['2021':'2021']], axis=0)
    else:
        previsoes_santarem_2002 = previsoes_santarem['2002':'2002']
        previsoes_santarem_2003 = previsoes_santarem['2003':'2003']
        previsoes_santarem_2004 = previsoes_santarem['2004':'2004']
        previsoes_santarem_2005 = previsoes_santarem['2005':'2005']
        previsoes_santarem_2006 = previsoes_santarem['2006':'2006']
        previsoes_santarem_2007 = previsoes_santarem['2007':'2007']
        previsoes_santarem_2008 = previsoes_santarem['2008':'2008']
        previsoes_santarem_2009 = previsoes_santarem['2009':'2009']
        previsoes_santarem_2010 = previsoes_santarem['2010':'2010']
        previsoes_santarem_2011 = previsoes_santarem['2011':'2011']
        previsoes_santarem_2012 = previsoes_santarem['2012':'2012']
        previsoes_santarem_2013 = previsoes_santarem['2013':'2013']
        previsoes_santarem_2014 = previsoes_santarem['2014':'2014']
        previsoes_santarem_2015 = previsoes_santarem['2015':'2015']
        previsoes_santarem_2016 = previsoes_santarem['2016':'2016']
        previsoes_santarem_2017 = previsoes_santarem['2017':'2017']
        previsoes_santarem_2018 = previsoes_santarem['2018':'2018']
        previsoes_santarem_2019 = previsoes_santarem['2019':'2019']
        previsoes_santarem_2020 = previsoes_santarem['2020':'2020']
        previsoes_santarem_2021 = previsoes_santarem['2021':'2021']

previsoes_santarem_diario = pd.concat([ previsoes_santarem_2002,
                                        previsoes_santarem_2003,
                                        previsoes_santarem_2004,
                                        previsoes_santarem_2005,
                                        previsoes_santarem_2006,
                                        previsoes_santarem_2007,
                                        previsoes_santarem_2008,
                                        previsoes_santarem_2009,
                                        previsoes_santarem_2010,
                                        previsoes_santarem_2011,
                                        previsoes_santarem_2012,
                                        previsoes_santarem_2013,
                                        previsoes_santarem_2014,
                                        previsoes_santarem_2015,
                                        previsoes_santarem_2016,
                                        previsoes_santarem_2017,
                                        previsoes_santarem_2018,
                                        previsoes_santarem_2019,
                                        previsoes_santarem_2020,
                                        previsoes_santarem_2021], axis=0)

previsoes_santarem_diario = ExponentialSmoothing(previsoes_santarem_diario, initialization_method="heuristic",freq='D').fit(optimized=True).fittedvalues


previsoes_santarem_diario = previsoes_santarem_diario.rename_axis('index')
previsoes_santarem_diario = previsoes_santarem_diario.rename('GPP')

previsoes_santarem_diario.to_csv('gpp_novo_2002_2021/previsoes_santarem_2002_2021.csv')

## Peru

In [8]:
def constructor_2(path):
    nome = path.split('/')[1].split('_')[0]
    df = pd.read_csv(path, parse_dates=['date'])
    df = df.rename({'albedo':nome},axis=1)
    # Definindo a coluna 'system:time_start' como índice
    hora_inicio = df['date'].min().floor('d')
    df.index = hora_inicio + pd.to_timedelta(df.index, unit='h')
    # Removendo a coluna 'system:time_start'
    df = df.drop(['date','.geo'], axis=1)
    df = df.drop('system:index',axis=1)
    return df

# Leitura e pré-processamento dos dados de temperatura
tday = constructor_2('dados/Merra/amazonia peruana/pe_t10m.csv') - 273.15 #( K° -> C°)
tday = tday['2002':'2021']
tmin = tday['T10M'].resample('D').min().values  # Reamostrando para Temperatura mínima diária
tmean_peru = tday['T10M'].resample('D').mean().values  # Temperatura média diária
tmean_vpd = tday['T10M'].between_time('10:00', '22:00').resample('D').mean().values

# Leitura e pré-processamento dos dados de razão de mistura de vapor de água
qv10m = constructor_2('dados/Merra/amazonia peruana/pe_qv10m.csv') # unidade: "Mass fraction" (kg/kg)
qv10m = qv10m['2002':'2021']
qv10m = pd.to_numeric(qv10m['QV10M']).between_time('10:00', '22:00').resample('D').mean().values

# Leitura e pré-processamento dos dados de pressão atmosférica
ps = constructor_2('dados/Merra/amazonia peruana/pe_ps.csv') # unidade: Pa  # Removendo vírgulas dos valores
ps = ps['2002':'2021']
ps['PS'] = pd.to_numeric(ps['PS'])
ps = ps.between_time('10:00', '22:00').resample('D').mean()
ps = pd.to_numeric(ps['PS']).values

# Leitura e pré-processamento dos dados de radiação solar incidente
SWGNT = constructor_2('dados/Merra/amazonia peruana/pe_swgnt.csv')
SWGNT = SWGNT['2002':'2021']
SWGNT = SWGNT['SWGNT'].resample('D').mean().values # unidade: W/m^2

# Função para converter datas no formato MODIS
def modis_date(data_str):
    ano = int(data_str[1:5])
    dia_ano = int(data_str[5:])
    data = datetime.strptime('{}-{}'.format(ano, dia_ano), '%Y-%j')
    return data

# Leitura e pré-processamento dos dados de Fração de Absorção de Luz Fotossinteticamente Ativa (FPAR)
fpar = pd.read_csv('dados/Merra/amazonia peruana/pe_fpar.csv', header=None, na_values='F') # unidade: (%)
fpar.index = fpar[2].apply(modis_date)  # Convertendo a data no formato MODIS

# Arrumando o dataframe
# ================================
fpar = fpar.drop([0,1,2,3,4], axis=1)
fpar.columns = range(len(fpar.columns))
fpar = fpar[144]

# ================================

# Tratamento de dados faltando usando interpolação linear (8 dias -> diário)
fpar = fpar.resample('D').interpolate('linear')['2002':'2021']

fpar = fpar.values

drivers_peru = [
    fpar[:][...,None],
    tmin[:][...,None],
    MOD17.vpd(qv10m, ps, tmean_vpd)[...,None],
    MOD17.par(SWGNT)[:][...,None]
]
drivers_peru[2] = np.where(drivers_peru[2] < 0, 0, drivers_peru[2]) # VPD < 0 = 0

dict = {'1': np.squeeze(drivers_peru[0]), '2': np.squeeze(drivers_peru[1]), '3': np.squeeze(drivers_peru[2]), '4': np.squeeze(drivers_peru[3])} 
index_peru = pd.date_range(start='2002-01-01', end='2021-12-31', freq='D')

df_drivers_peru = pd.DataFrame(dict, index=index_peru)

drivers_peru_mes = []
for i in range(12):
    mes = df_drivers_peru[df_drivers_peru.index.month == i+1].to_numpy().T
    drivers_peru_mes.append(mes)

bplut_mensal_peru = pd.read_csv('GA_Peru.csv')
bplut_mensal_peru = bplut_mensal_peru[['LUE_Max', 'Tmin_min', 'Tmin_max', 'VPD_min', 'VPD_max']].to_numpy()


for i in range(12):
    previsoes_peru = MOD17._gpp(bplut_mensal_peru[i], drivers_peru_mes[i][0], drivers_peru_mes[i][1], drivers_peru_mes[i][2], drivers_peru_mes[i][3])
    previsoes_peru = pd.DataFrame(previsoes_peru,index=index_peru[index_peru.month == i+1])
    
   

    #previsoes_peru_pso = pd.concat([gpp_kgc_8d_2009_teste[ano2009],gpp_kgc_8d_2010_teste[ano2010],gpp_kgc_8d_2011_teste[ano2011]],axis=0)
    if i > 0:
        previsoes_peru_2002 = pd.concat([previsoes_peru_2002, previsoes_peru['2002':'2002']], axis=0)
        previsoes_peru_2003 = pd.concat([previsoes_peru_2003, previsoes_peru['2003':'2003']], axis=0)
        previsoes_peru_2004 = pd.concat([previsoes_peru_2004, previsoes_peru['2004':'2004']], axis=0)
        previsoes_peru_2005 = pd.concat([previsoes_peru_2005, previsoes_peru['2005':'2005']], axis=0)
        previsoes_peru_2006 = pd.concat([previsoes_peru_2006, previsoes_peru['2006':'2006']], axis=0)
        previsoes_peru_2007 = pd.concat([previsoes_peru_2007, previsoes_peru['2007':'2007']], axis=0)
        previsoes_peru_2008 = pd.concat([previsoes_peru_2008, previsoes_peru['2008':'2008']], axis=0)
        previsoes_peru_2009 = pd.concat([previsoes_peru_2009, previsoes_peru['2009':'2009']], axis=0)
        previsoes_peru_2010 = pd.concat([previsoes_peru_2010, previsoes_peru['2010':'2010']], axis=0)
        previsoes_peru_2011 = pd.concat([previsoes_peru_2011, previsoes_peru['2011':'2011']], axis=0)
        previsoes_peru_2012 = pd.concat([previsoes_peru_2012, previsoes_peru['2012':'2012']], axis=0)
        previsoes_peru_2013 = pd.concat([previsoes_peru_2013, previsoes_peru['2013':'2013']], axis=0)
        previsoes_peru_2014 = pd.concat([previsoes_peru_2014, previsoes_peru['2014':'2014']], axis=0)
        previsoes_peru_2015 = pd.concat([previsoes_peru_2015, previsoes_peru['2015':'2015']], axis=0)
        previsoes_peru_2016 = pd.concat([previsoes_peru_2016, previsoes_peru['2016':'2016']], axis=0)
        previsoes_peru_2017 = pd.concat([previsoes_peru_2017, previsoes_peru['2017':'2017']], axis=0)
        previsoes_peru_2018 = pd.concat([previsoes_peru_2018, previsoes_peru['2018':'2018']], axis=0)
        previsoes_peru_2019 = pd.concat([previsoes_peru_2019, previsoes_peru['2019':'2019']], axis=0)
        previsoes_peru_2020 = pd.concat([previsoes_peru_2020, previsoes_peru['2020':'2020']], axis=0)
        previsoes_peru_2021 = pd.concat([previsoes_peru_2021, previsoes_peru['2021':'2021']], axis=0)
    else:
        previsoes_peru_2002 = previsoes_peru['2002':'2002']
        previsoes_peru_2003 = previsoes_peru['2003':'2003']
        previsoes_peru_2004 = previsoes_peru['2004':'2004']
        previsoes_peru_2005 = previsoes_peru['2005':'2005']
        previsoes_peru_2006 = previsoes_peru['2006':'2006']
        previsoes_peru_2007 = previsoes_peru['2007':'2007']
        previsoes_peru_2008 = previsoes_peru['2008':'2008']
        previsoes_peru_2009 = previsoes_peru['2009':'2009']
        previsoes_peru_2010 = previsoes_peru['2010':'2010']
        previsoes_peru_2011 = previsoes_peru['2011':'2011']
        previsoes_peru_2012 = previsoes_peru['2012':'2012']
        previsoes_peru_2013 = previsoes_peru['2013':'2013']
        previsoes_peru_2014 = previsoes_peru['2014':'2014']
        previsoes_peru_2015 = previsoes_peru['2015':'2015']
        previsoes_peru_2016 = previsoes_peru['2016':'2016']
        previsoes_peru_2017 = previsoes_peru['2017':'2017']
        previsoes_peru_2018 = previsoes_peru['2018':'2018']
        previsoes_peru_2019 = previsoes_peru['2019':'2019']
        previsoes_peru_2020 = previsoes_peru['2020':'2020']
        previsoes_peru_2021 = previsoes_peru['2021':'2021']

previsoes_peru_diario = pd.concat([ previsoes_peru_2002,
                                        previsoes_peru_2003,
                                        previsoes_peru_2004,
                                        previsoes_peru_2005,
                                        previsoes_peru_2006,
                                        previsoes_peru_2007,
                                        previsoes_peru_2008,
                                        previsoes_peru_2009,
                                        previsoes_peru_2010,
                                        previsoes_peru_2011,
                                        previsoes_peru_2012,
                                        previsoes_peru_2013,
                                        previsoes_peru_2014,
                                        previsoes_peru_2015,
                                        previsoes_peru_2016,
                                        previsoes_peru_2017,
                                        previsoes_peru_2018,
                                        previsoes_peru_2019,
                                        previsoes_peru_2020,
                                        previsoes_peru_2021], axis=0)


previsoes_peru_diario = ExponentialSmoothing(previsoes_peru_diario, initialization_method="heuristic",freq='D').fit(optimized=True).fittedvalues

previsoes_peru_diario = previsoes_peru_diario.rename_axis('index')
previsoes_peru_diario = previsoes_peru_diario.rename('GPP')

previsoes_peru_diario.to_csv('gpp_novo_2002_2021/previsoes_peru_2002_2021.csv')


## Caxiuana

In [9]:
def constructor_2(path):
    nome = path.split('/')[1].split('_')[0]
    df = pd.read_csv(path, parse_dates=['date'])
    df = df.rename({'albedo':nome},axis=1)
    # Definindo a coluna 'system:time_start' como índice
    hora_inicio = df['date'].min().floor('d')
    df.index = hora_inicio + pd.to_timedelta(df.index, unit='h')
    # Removendo a coluna 'system:time_start'
    df = df.drop(['date','.geo'], axis=1)
    df = df.drop('system:index',axis=1)
    return df

# Leitura e pré-processamento dos dados de temperatura
tday = constructor_2('dados/Merra/caxiuana/cax_t10m.csv') - 273.15 #( K° -> C°)
tday = tday['2002':'2021']
tmin = tday['T10M'].resample('D').min().values  # Reamostrando para Temperatura mínima diária
tmean_caxiuana = tday['T10M'].resample('D').mean().values  # Temperatura média diária
tmean_vpd = tday['T10M'].between_time('10:00', '22:00').resample('D').mean().values

# Leitura e pré-processamento dos dados de razão de mistura de vapor de água
qv10m = constructor_2('dados/Merra/caxiuana/cax_qv10m.csv') # unidade: "Mass fraction" (kg/kg)
qv10m = qv10m['2002':'2021']
qv10m = pd.to_numeric(qv10m['QV10M']).between_time('10:00', '22:00').resample('D').mean().values

# Leitura e pré-processamento dos dados de pressão atmosférica
ps = constructor_2('dados/Merra/caxiuana/cax_ps.csv') # unidade: Pa  # Removendo vírgulas dos valores
ps = ps['2002':'2021']
ps['PS'] = pd.to_numeric(ps['PS'])
ps = ps.between_time('10:00', '22:00').resample('D').mean()
ps = pd.to_numeric(ps['PS']).values

# Leitura e pré-processamento dos dados de radiação solar incidente
SWGNT = constructor_2('dados/Merra/caxiuana/cax_swgnt.csv')
SWGNT = SWGNT['2002':'2021']
SWGNT = SWGNT['SWGNT'].resample('D').mean().values # unidade: W/m^2

# Função para converter datas no formato MODIS
def modis_date(data_str):
    ano = int(data_str[1:5])
    dia_ano = int(data_str[5:])
    data = datetime.strptime('{}-{}'.format(ano, dia_ano), '%Y-%j')
    return data

# Leitura e pré-processamento dos dados de Fração de Absorção de Luz Fotossinteticamente Ativa (FPAR)
fpar = pd.read_csv('dados/Merra/caxiuana/cax_fpar.csv', header=None, na_values='F') # unidade: (%)
fpar.index = fpar[2].apply(modis_date)  # Convertendo a data no formato MODIS

# Arrumando o dataframe
# ================================
fpar = fpar.drop([0,1,2,3,4], axis=1)
fpar.columns = range(len(fpar.columns))
fpar = fpar[144]

# ================================

# Tratamento de dados faltando usando interpolação linear (8 dias -> diário)
fpar = fpar.resample('D').interpolate('linear')['2002':'2021']

fpar = fpar.values

drivers_caxiuana = [
    fpar[:][...,None],
    tmin[:][...,None],
    MOD17.vpd(qv10m, ps, tmean_vpd)[...,None],
    MOD17.par(SWGNT)[:][...,None]
]
drivers_caxiuana[2] = np.where(drivers_caxiuana[2] < 0, 0, drivers_caxiuana[2]) # VPD < 0 = 0

dict = {'1': np.squeeze(drivers_caxiuana[0]), '2': np.squeeze(drivers_caxiuana[1]), '3': np.squeeze(drivers_caxiuana[2]), '4': np.squeeze(drivers_caxiuana[3])} 
index_caxiuana = pd.date_range(start='2002-01-01', end='2021-12-31', freq='D')

df_drivers_caxiuana = pd.DataFrame(dict, index=index_caxiuana)

drivers_caxiuana_mes = []
for i in range(12):
    mes = df_drivers_caxiuana[df_drivers_caxiuana.index.month == i+1].to_numpy().T
    drivers_caxiuana_mes.append(mes)

bplut_mensal_caxiuana = pd.read_csv('GA_Caxiuana.csv')
bplut_mensal_caxiuana = bplut_mensal_caxiuana[['LUE_Max', 'Tmin_min', 'Tmin_max', 'VPD_min', 'VPD_max']].to_numpy()


for i in range(12):
    previsoes_caxiuana = MOD17._gpp(bplut_mensal_caxiuana[i], drivers_caxiuana_mes[i][0], drivers_caxiuana_mes[i][1], drivers_caxiuana_mes[i][2], drivers_caxiuana_mes[i][3])
    previsoes_caxiuana = pd.DataFrame(previsoes_caxiuana,index=index_caxiuana[index_caxiuana.month == i+1])
    
   

    #previsoes_caxiuana_pso = pd.concat([gpp_kgc_8d_2009_teste[ano2009],gpp_kgc_8d_2010_teste[ano2010],gpp_kgc_8d_2011_teste[ano2011]],axis=0)
    if i > 0:
        previsoes_caxiuana_2002 = pd.concat([previsoes_caxiuana_2002, previsoes_caxiuana['2002':'2002']], axis=0)
        previsoes_caxiuana_2003 = pd.concat([previsoes_caxiuana_2003, previsoes_caxiuana['2003':'2003']], axis=0)
        previsoes_caxiuana_2004 = pd.concat([previsoes_caxiuana_2004, previsoes_caxiuana['2004':'2004']], axis=0)
        previsoes_caxiuana_2005 = pd.concat([previsoes_caxiuana_2005, previsoes_caxiuana['2005':'2005']], axis=0)
        previsoes_caxiuana_2006 = pd.concat([previsoes_caxiuana_2006, previsoes_caxiuana['2006':'2006']], axis=0)
        previsoes_caxiuana_2007 = pd.concat([previsoes_caxiuana_2007, previsoes_caxiuana['2007':'2007']], axis=0)
        previsoes_caxiuana_2008 = pd.concat([previsoes_caxiuana_2008, previsoes_caxiuana['2008':'2008']], axis=0)
        previsoes_caxiuana_2009 = pd.concat([previsoes_caxiuana_2009, previsoes_caxiuana['2009':'2009']], axis=0)
        previsoes_caxiuana_2010 = pd.concat([previsoes_caxiuana_2010, previsoes_caxiuana['2010':'2010']], axis=0)
        previsoes_caxiuana_2011 = pd.concat([previsoes_caxiuana_2011, previsoes_caxiuana['2011':'2011']], axis=0)
        previsoes_caxiuana_2012 = pd.concat([previsoes_caxiuana_2012, previsoes_caxiuana['2012':'2012']], axis=0)
        previsoes_caxiuana_2013 = pd.concat([previsoes_caxiuana_2013, previsoes_caxiuana['2013':'2013']], axis=0)
        previsoes_caxiuana_2014 = pd.concat([previsoes_caxiuana_2014, previsoes_caxiuana['2014':'2014']], axis=0)
        previsoes_caxiuana_2015 = pd.concat([previsoes_caxiuana_2015, previsoes_caxiuana['2015':'2015']], axis=0)
        previsoes_caxiuana_2016 = pd.concat([previsoes_caxiuana_2016, previsoes_caxiuana['2016':'2016']], axis=0)
        previsoes_caxiuana_2017 = pd.concat([previsoes_caxiuana_2017, previsoes_caxiuana['2017':'2017']], axis=0)
        previsoes_caxiuana_2018 = pd.concat([previsoes_caxiuana_2018, previsoes_caxiuana['2018':'2018']], axis=0)
        previsoes_caxiuana_2019 = pd.concat([previsoes_caxiuana_2019, previsoes_caxiuana['2019':'2019']], axis=0)
        previsoes_caxiuana_2020 = pd.concat([previsoes_caxiuana_2020, previsoes_caxiuana['2020':'2020']], axis=0)
        previsoes_caxiuana_2021 = pd.concat([previsoes_caxiuana_2021, previsoes_caxiuana['2021':'2021']], axis=0)
    else:
        previsoes_caxiuana_2002 = previsoes_caxiuana['2002':'2002']
        previsoes_caxiuana_2003 = previsoes_caxiuana['2003':'2003']
        previsoes_caxiuana_2004 = previsoes_caxiuana['2004':'2004']
        previsoes_caxiuana_2005 = previsoes_caxiuana['2005':'2005']
        previsoes_caxiuana_2006 = previsoes_caxiuana['2006':'2006']
        previsoes_caxiuana_2007 = previsoes_caxiuana['2007':'2007']
        previsoes_caxiuana_2008 = previsoes_caxiuana['2008':'2008']
        previsoes_caxiuana_2009 = previsoes_caxiuana['2009':'2009']
        previsoes_caxiuana_2010 = previsoes_caxiuana['2010':'2010']
        previsoes_caxiuana_2011 = previsoes_caxiuana['2011':'2011']
        previsoes_caxiuana_2012 = previsoes_caxiuana['2012':'2012']
        previsoes_caxiuana_2013 = previsoes_caxiuana['2013':'2013']
        previsoes_caxiuana_2014 = previsoes_caxiuana['2014':'2014']
        previsoes_caxiuana_2015 = previsoes_caxiuana['2015':'2015']
        previsoes_caxiuana_2016 = previsoes_caxiuana['2016':'2016']
        previsoes_caxiuana_2017 = previsoes_caxiuana['2017':'2017']
        previsoes_caxiuana_2018 = previsoes_caxiuana['2018':'2018']
        previsoes_caxiuana_2019 = previsoes_caxiuana['2019':'2019']
        previsoes_caxiuana_2020 = previsoes_caxiuana['2020':'2020']
        previsoes_caxiuana_2021 = previsoes_caxiuana['2021':'2021']

previsoes_caxiuana_diario = pd.concat([ previsoes_caxiuana_2002,
                                        previsoes_caxiuana_2003,
                                        previsoes_caxiuana_2004,
                                        previsoes_caxiuana_2005,
                                        previsoes_caxiuana_2006,
                                        previsoes_caxiuana_2007,
                                        previsoes_caxiuana_2008,
                                        previsoes_caxiuana_2009,
                                        previsoes_caxiuana_2010,
                                        previsoes_caxiuana_2011,
                                        previsoes_caxiuana_2012,
                                        previsoes_caxiuana_2013,
                                        previsoes_caxiuana_2014,
                                        previsoes_caxiuana_2015,
                                        previsoes_caxiuana_2016,
                                        previsoes_caxiuana_2017,
                                        previsoes_caxiuana_2018,
                                        previsoes_caxiuana_2019,
                                        previsoes_caxiuana_2020,
                                        previsoes_caxiuana_2021], axis=0)

previsoes_caxiuana_diario = ExponentialSmoothing(previsoes_caxiuana_diario, initialization_method="heuristic",freq='D').fit(optimized=True).fittedvalues

previsoes_caxiuana_diario = previsoes_caxiuana_diario.rename_axis('index')
previsoes_caxiuana_diario = previsoes_caxiuana_diario.rename('GPP')

previsoes_caxiuana_diario.to_csv('gpp_novo_2002_2021/previsoes_caxiuana_2002_2021.csv')
